In [0]:
#!apt-get install -y xvfb python-opengl > /dev/null 2>&1
#!pip install gym pyvirtualdisplay > /dev/null 2>&1
#!apt-get install x11-utils
!pip install keras-rl
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.13.1 

     |████████████████████████████████| 40kB 3.1MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=15a7bf1c08c1d20ea19a4d341395a8e4e10b7a2d05bc6bbfb4eb92ca61ca83f5
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 4.9MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 3.2MB 36.7MB/s 
     |████████████████████████████████| 368kB 49.9MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.15.0
 

In [1]:
from __future__ import division
import gym
import atari_py
from PIL import Image
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from keras.optimizers import Adam
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
#print(atari_py.list_games())

print(keras.__version__)
print(tf.__version__)


Using TensorFlow backend.


2.2.4
1.13.1


In [2]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('float32')/255 

    #def process_reward(self, reward):
     #   return np.clip(reward, -1., 1.)

In [3]:
game = 'Centipede-v0' #18
#game = 'Breakout-v0' #4
#game = 'Solaris-v0' #18
#game = 'DemonAttack-v0' #6
#game = 'AirRaid-v0' #6

env = gym.make(game, full_action_space=True)
np.random.seed(123)
env.seed(123)
#env.reset()
nb_actions = env.action_space.n
#print(nb_actions)

In [4]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE
model = Sequential()
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    model.add(Permute((2, 3, 1), input_shape=input_shape))
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    model.add(Permute((1, 2, 3), input_shape=input_shape))
else:
    raise RuntimeError('Unknown image_dim_ordering.')

model.add(Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu'))
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dense(nb_actions, activation = 'linear'))

print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               1606144   
_________________________________________________________________
dens

In [5]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()

policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=100000)

dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn.compile(Adam(lr=.00025), metrics=['mae'])

In [0]:
dqn.load_weights('DQN_AirRaid-v0_weights')
dqn.fit(env, nb_steps=200000, visualize=False, verbose=1)
dqn.save_weights('DQN_'+game+'_weights',overwrite=True)

Training for 200000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 28s 3ms/step - reward: 2.0996
11 episodes - episode_reward: 1897.455 [844.000, 3723.000] - ale.lives: 2.106

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.0671
11 episodes - episode_reward: 1852.909 [561.000, 3078.000] - ale.lives: 2.006

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.1161
11 episodes - episode_reward: 1940.091 [428.000, 3712.000] - ale.lives: 2.041

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 1.7302
11 episodes - episode_reward: 1584.091 [598.000, 2681.000] - ale.lives: 2.166

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 27s 3ms/step - reward: 2.4528
8 episodes - episode_reward: 2555.875 [1561.000, 5075.000] - ale.lives: 2.115

Interval 6 (50000 s

In [8]:
dqn.load_weights('DQN_'+game+'_weights')
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...
Episode 1: reward: 4035.000, steps: 1205
Episode 2: reward: 8873.000, steps: 1478
Episode 3: reward: 1246.000, steps: 587
Episode 4: reward: 7139.000, steps: 1598
Episode 5: reward: 7153.000, steps: 1343
Episode 6: reward: 2787.000, steps: 1031
Episode 7: reward: 1619.000, steps: 896
Episode 8: reward: 3605.000, steps: 1132
Episode 9: reward: 5453.000, steps: 886
Episode 10: reward: 2757.000, steps: 901


In [9]:
env.close()

In [10]:
dqn.load_weights('DQN_'+game+'_weights')
history = dqn.test(env, nb_episodes=100, visualize=False)

from statistics import mean

print('Average Reward : {}'.format(mean(history.history['episode_reward'])))

Testing for 100 episodes ...
Episode 1: reward: 1136.000, steps: 679
Episode 2: reward: 5294.000, steps: 1395
Episode 3: reward: 3318.000, steps: 1219
Episode 4: reward: 1553.000, steps: 670
Episode 5: reward: 2438.000, steps: 1111
Episode 6: reward: 2520.000, steps: 1085
Episode 7: reward: 2113.000, steps: 1095
Episode 8: reward: 2306.000, steps: 1149
Episode 9: reward: 1457.000, steps: 846
Episode 10: reward: 2514.000, steps: 659
Episode 11: reward: 3946.000, steps: 1573
Episode 12: reward: 2437.000, steps: 1020
Episode 13: reward: 1359.000, steps: 657
Episode 14: reward: 1841.000, steps: 665
Episode 15: reward: 6387.000, steps: 1416
Episode 16: reward: 4554.000, steps: 1523
Episode 17: reward: 4760.000, steps: 1616
Episode 18: reward: 3800.000, steps: 887
Episode 19: reward: 1434.000, steps: 1260
Episode 20: reward: 2358.000, steps: 666
Episode 21: reward: 5754.000, steps: 1287
Episode 22: reward: 2744.000, steps: 974
Episode 23: reward: 1599.000, steps: 1068
Episode 24: reward: 794